# Variational Autoencoder


## 실습 목표

- VAE를 설계하고 학습시켜 이미지를 생성하는 모델을 만듭니다. (데이터셋: MNIST)


## 문제 정의

- VAE

![image](https://upload.wikimedia.org/wikipedia/commons/1/11/Reparameterized_Variational_Autoencoder.png)

## VAE

### [Step1] Load libraries & Datasets

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from torchvision.transforms import ToTensor

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torch.nn as nn
import torch.nn.functional as F
import torch

In [4]:
train_images = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor() 
)

test_images = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

12.2%

100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100.0%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100.0%


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100.0%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



### [Step2] Set hyperparameters

In [5]:
# Hyperparameters
EPOCH = 10
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
DEVICE = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
print('Using Device :', DEVICE)

Using Device : mps:0


### [Step3] Create DataLoader

In [6]:
train_loader = DataLoader(train_images, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_images, batch_size=BATCH_SIZE, shuffle=True)

### [Step4] Set Network Structure ([구조](https://upload.wikimedia.org/wikipedia/commons/1/11/Reparameterized_Variational_Autoencoder.png))

In [7]:
class VAE(nn.Module):
    def __init__(self, latent_dim):     # latent_dim : 잠재 변수의 차원
        super(VAE, self).__init__()
        self.flatten = nn.Flatten()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(256, latent_dim)    # 평균, latent_dim 차원
        self.fc_var = nn.Linear(256, latent_dim)   # 분산, latent_dim 차원
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 28 * 28),
            nn.Sigmoid(),
        )

    def encode(self, x):
        result = self.encoder(x)
        mu = self.fc_mu(result)
        var = self.fc_var(result)
        return mu, var
    
    def decode(self, z):
        result = self.decoder(z)
        return result
    
    def reparameterize(self, mu, var):
        std = torch.exp(var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        x = self.flatten(x)
        mu, var = self.encode(x)
        z = self.reparameterize(mu, var)
        out = self.decode(z)
        return out, mu, var

### [Step5] Create Model instance

In [8]:
model = VAE(10).to(DEVICE)
print(model)

VAE(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
  )
  (fc_mu): Linear(in_features=256, out_features=10, bias=True)
  (fc_var): Linear(in_features=256, out_features=10, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=784, bias=True)
    (5): Sigmoid()
  )
)


### [Step6] Model compile

In [9]:
def loss_function(recon_x, x, mu, var):
    recon_loss = F.binary_cross_entropy(recon_x, x.view(-1, 28 * 28), reduction='sum')  # reconstruction loss
    kl_loss = -0.5 * torch.sum(1 + var - mu.pow(2) - var.exp())    # KL divergence loss
    return recon_loss + kl_loss

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

### [Step6] Model compile